In [1]:
!pip install pandas
!pip install torch
!pip install nltk
!pip install tqdm
!pip install seaborn
!pip install numpy
!pip install sklearn

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Скачиваем данные

In [3]:
!wget https://raw.githubusercontent.com/semensorokin/DLforNLP_course_material/master/Homework2/answers_subsample.csv

--2021-12-21 18:33:15--  https://raw.githubusercontent.com/semensorokin/DLforNLP_course_material/master/Homework2/answers_subsample.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28717126 (27M) [text/plain]
Saving to: ‘answers_subsample.csv’

answers_subsample.c 100%[===================>]  27.39M   183MB/s    in 0.2s    

2021-12-21 18:33:16 (183 MB/s) - ‘answers_subsample.csv’ saved [28717126/28717126]



In [ ]:
# если ругается на то, что нет wget
# !apt-get install wget

In [ ]:
!ls -l

total 28052
-rw-r--r-- 1 root root 28717126 Dec 21 13:42 answers_subsample.csv
drwxr-xr-x 1 root root     4096 Dec  3 14:33 sample_data


In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('answers_subsample.csv')

In [6]:
data

,category,text
0,business,Могут ли в россельхозбанке дать в залог норков...
1,law,Может ли срочник перевестись на контракт после...
2,business,Продажа недвижимости по ипотеки ? ( арестованы...
3,business,"В чем смысл криптовалюты, какая от неё выгода ..."
4,law,часть 1 статья 158 похитил телефон
...,...,...
237774,relax,елку нарядили? =)
237775,law,"Имеется переработка при 75% ставки, отгулы не ..."
237776,food,Попробовала варить рис с половиной кубика для ...
237777,food,Почему рекоменд... Почему рекомендуют есть фру...


In [7]:
data.category.value_counts() * 100 / data.shape[0]

law         29.793211
relax       22.016242
business    19.309527
food        18.367055
love        10.513965
Name: category, dtype: float64

# Предобученные эмбеддинги
[Источник](https://fasttext.cc/docs/en/crawl-vectors.html)  
Вы можете взять любые word2vec подобные эмббединги. Если вы хотите использовать elmo, bert, etc сначала попробуйте с word2vec подобными эмббедингами, а потом можете перейти к более сложным моделям.  
Ниже мы сначала скачиваем, а потом распоковываем эмбеддинги.

In [70]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
!gzip -d cc.ru.300.vec.gz

--2021-12-21 19:23:45--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1306357571 (1.2G) [binary/octet-stream]
Saving to: ‘cc.ru.300.vec.gz’

cc.ru.300.vec.gz    100%[===================>]   1.22G  43.2MB/s    in 28s     

2021-12-21 19:24:14 (43.8 MB/s) - ‘cc.ru.300.vec.gz’ saved [1306357571/1306357571]



In [71]:
!ls -l

total 6978728
-rw-r--r-- 1 root root   28717126 Dec 21 18:33 answers_subsample.csv
-rw-r--r-- 1 root root 4536408847 Jan 18  2019 cc.ru.300.vec
-rw-r--r-- 1 root root 2581071412 Jan 11  2018 ft_native_300_ru_wiki_lenta_lemmatize.vec
drwxr-xr-x 1 root root       4096 Dec  3 14:33 sample_data


In [72]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize
import pymorphy2
from tqdm import tqdm
from string import punctuation
from nltk.corpus import stopwords

In [73]:
stops = stopwords.words("russian")
stops.extend(list(punctuation))

Изначально ради интереса я пытался использовать эмбеддинги лемм (и лемматизировать тексты тоже, соответственно), но это сильно понижало качество, поэтому я использовал дефолтную модель.

In [74]:
# потом можете добавить свою предобработку

morph = pymorphy2.MorphAnalyzer()

def process_text(text):
    
    words = [token for token in word_tokenize(text.lower()) if token not in stops]
    
    return words

In [75]:
word2freq = {}
lengths = []

for text in tqdm(data.text):
    
    words = process_text(text)
    
    lengths.append(len(words))
    
    for word in words:
        
        if word in word2freq:
            word2freq[word] += 1
        else:
            word2freq[word] = 1

100%|██████████| 237779/237779 [00:40<00:00, 5815.78it/s]


In [76]:
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(16, 10))
plt.title('Распределение длин предложений в словах')
plt.xlabel('Длина предложения')
plt.ylabel('Доля')
sns.distplot(lengths)

In [79]:
upper_threshold = 32
lower_threshold = 3

correct_percent = len([sent_len for sent_len in lengths 
                       if sent_len <= upper_threshold and sent_len >= lower_threshold]) * 100 / len(lengths)

'{:.2f} % наших текстов входят в промежуток от {} до {} слов'.format(correct_percent, lower_threshold, upper_threshold)

'93.42 % наших текстов входят в промежуток от 3 до 32 слов'

In [80]:
len(word2freq)

164964

In [81]:
'{} слов, которые встречались 3 и менее раз'.format(len([word for word in word2freq if word2freq[word] <= 3]))

'127547 слов, которые встречались 3 и менее раз'

# Читаем файл с эмбеддингами
### Этот файл с 300 числами для 2 000 000 слов и он может не влезть в память
Поэтому прочитаем только те слова, которые мы знаем

In [82]:
import numpy as np

In [83]:
word2index = {'PAD': 0}
vectors = []
    
word2vec_file = open('cc.ru.300.vec') 

n_words, embedding_dim = word2vec_file.readline().split()
n_words, embedding_dim = int(n_words), int(embedding_dim)

# Zero vector for PAD
vectors.append(np.zeros((1, embedding_dim)))

progress_bar = tqdm(desc='Read word2vec', total=n_words)

while True:

    line = word2vec_file.readline().strip()

    if not line:
        break
        
    current_parts = line.split()

    current_word = ' '.join(current_parts[:-embedding_dim])

    if current_word in word2freq:

        word2index[current_word] = len(word2index)

        current_vectors = current_parts[-embedding_dim:]
        current_vectors = np.array(list(map(float, current_vectors)))
        current_vectors = np.expand_dims(current_vectors, 0)

        vectors.append(current_vectors)

    progress_bar.update(1)

progress_bar.close()

word2vec_file.close()

vectors = np.concatenate(vectors)

Read word2vec: 100%|██████████| 2000000/2000000 [00:55<00:00, 35742.65it/s]


In [84]:
len(word2index)

119903

In [85]:
unk_words = [word for word in word2freq if word not in word2index]
unk_counts = [word2freq[word] for word in unk_words]
n_unk = sum(unk_counts) * 100 / sum(list(word2freq.values()))

sub_sample_unk_words = {word: word2freq[word] for word in unk_words}
sorted_unk_words = list(sorted(sub_sample_unk_words, key=lambda x: sub_sample_unk_words[x], reverse=True))

print('Мы не знаем {:.2f} % слов в датасете'.format(n_unk))
print('Количество неизвестных слов {} из {}, то есть {:.2f} % уникальных слов в словаре'.format(
    len(unk_words), len(word2freq), len(unk_words) * 100 / len(word2freq)))
print('В среднем каждое встречается {:.2f} раз'.format(np.mean(unk_counts)))
print()
print('Топ 5 невошедших слов:')

for i in range(5):
    print(sorted_unk_words[i], 'с количеством вхождениий -', word2freq[sorted_unk_words[i]])

Мы не знаем 4.45 % слов в датасете
Количество неизвестных слов 45062 из 164964, то есть 27.32 % уникальных слов в словаре
В среднем каждое встречается 1.53 раз

Топ 5 невошедших слов:
'' с количеством вхождениий - 7907
`` с количеством вхождениий - 7812
+++ с количеством вхождениий - 91
16+ с количеством вхождениий - 87
з/п с количеством вхождениий - 80


# Потеря 2.5 % слов в датасете
Эта ситуация не то, чтобы сильно плохая, в учебных целях нормально, к тому же в среднем они редко встречаются. Вы можете поиграть с предобработкой.

In [86]:
import torch

- 128 - размер батча
- 64 - количество слов
- 1024 - эмбеддинг слова

In [ ]:
x = torch.rand(128, 64, 1024)

In [ ]:
lstm = torch.nn.LSTM(1024, 512, batch_first=True)

In [ ]:
%%timeit

pred = lstm(x)

1 loop, best of 5: 810 ms per loop


# А что GPU?

In [ ]:
print('Доступна ли видеокарта:', torch.cuda.is_available())
print('Если недоступна, поменяйте runtime, если в колабе')

Доступна ли видеокарта: True
Если недоступна, поменяйте runtime, если в колабе


In [ ]:
# универсальных способ задать device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# если доступна gpu, то давайте ее использовать, но в этом задании должны использовать

In [ ]:
# перенесли x на gpu
x_gpu = x.to(device)

In [ ]:
# зададим lstm на gpu
lstm_gpu = torch.nn.LSTM(1024, 512, batch_first=True)
lstm_gpu = lstm_gpu.to(device)

In [ ]:
%%timeit

pred = lstm_gpu(x_gpu)

100 loops, best of 5: 30.9 ms per loop


# У меня на 1070 TI скорость уменьшилась с 381мс до 41мс, то есть в 9.29 раз

In [ ]:
# если у нас модель на гпу, а то, что мы туда подаем нет, то работать не будет
# справедлива и обратная ситуация

# выскочит ошибка
# посмотрите на нее, возможно, вы еще встретитесь
# pred = lstm_gpu(x)

# Важные и не очень интуитивные моменты про LSTM и CNN в торче

По умолчанию LSTM принимает данные с такой размерностью:
```python
(seq_len, batch, input_size)
```
Сделано это с целью оптимизации на более низком уровне.  
Мы оперируем такими объектами:
```python
(batch, seq_len, input_size)
```
Чтобы LSTM у нас заработала правильно, мы можем либо передать параметр ```batch_first=True``` во время инициализации слоя,
либо транспонировать (поменять) первую и вторую размерность у нашего x перед подачей в слой.  
[Подробнее про LSTM](https://pytorch.org/docs/stable/nn.html#lstm)

- 128 - размер батча
- 64 - количество слов
- 1024 - эмбеддинг слова

In [ ]:
# первый способ
lstm = torch.nn.LSTM(1024, 512, batch_first=True)

pred, mem = lstm(x)

In [ ]:
pred.shape

torch.Size([128, 64, 512])

In [ ]:
lstm = torch.nn.LSTM(1024, 512)

# меняем размерность batch и seq_len местами
x_transposed = x.transpose(0, 1)
pred_transposed, mem = lstm(x_transposed)

In [ ]:
# у нас все еще осталась размерность (seq_len, batch, input_size)
pred_transposed.shape

torch.Size([64, 128, 512])

In [ ]:
# просто транспонируем еще раз
pred = pred_transposed.transpose(0, 1)
pred.shape

torch.Size([128, 64, 512])

## Conv1d & MaxPool1d
Примерно такая же ситуация происходит со сверточными слоями и пулингами.  
1d реализация как раз для текстов, в ней матрица-фильтр ходит только по одной размерности.  
[Подробнее про CNN](https://pytorch.org/docs/stable/nn.html#conv1d)  
[Подробнее про пулинг](https://pytorch.org/docs/stable/nn.html#maxpool1d)  
Ожидается такая размерность:
```python
(batch, input_size, seq_len)
```
Мы все еще хоти подавать такую размерность:
```python
(batch, seq_len, input_size)
```
В случае со свертками и пулингами у нас есть вариант только транспонировать x перед подачей и транспонировать полученный результат. Обратите внимание, что транспонируем мы первую и вторую размерность (индексация с нуля).

In [ ]:
x.shape

torch.Size([128, 64, 1024])

- 128 - размер батча
- 64 - количество слов
- 1024 - эмбеддинг слова

In [ ]:
# in_channels - размер входных эмбеддингов
# out_channels - количество/какой размер эмбеддингов мы хотим получить
# kernel_size - размер окна/н-граммы
cnn = torch.nn.Conv1d(in_channels=1024, out_channels=512, kernel_size=3)

In [ ]:
# выпадет ошибка, посмотрите какая
# pred = cnn(x)

In [ ]:
x_transposed = x.transpose(1, 2)
x_transposed.shape
# перевели в (batch, input_size, seq_len)

torch.Size([128, 1024, 64])

In [ ]:
pred_transposed = cnn(x_transposed)
pred_transposed.shape
# осталась разрмерность (batch, output_size, seq_len)

torch.Size([128, 512, 62])

In [ ]:
# переведем обратно в (batch, seq_len, input_size)
pred = pred_transposed.transpose(1, 2)
pred.shape

torch.Size([128, 62, 512])

# Подготовим данные в DataLoader

In [87]:
from torch.utils.data import Dataset, DataLoader

In [88]:
'UNK' in word2index

False

In [89]:
data.head()

,category,text
0,0,Могут ли в россельхозбанке дать в залог норков...
1,1,Может ли срочник перевестись на контракт после...
2,0,Продажа недвижимости по ипотеки ? ( арестованы...
3,0,"В чем смысл криптовалюты, какая от неё выгода ..."
4,1,часть 1 статья 158 похитил телефон


# Замапим категории в индексы

In [90]:
cat_mapper = {cat: n for n, cat in enumerate(data.category.unique())}

In [91]:
cat_mapper

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4}

In [92]:
data.category = data.category.map(cat_mapper)

In [93]:
data

,category,text
0,0,Могут ли в россельхозбанке дать в залог норков...
1,1,Может ли срочник перевестись на контракт после...
2,0,Продажа недвижимости по ипотеки ? ( арестованы...
3,0,"В чем смысл криптовалюты, какая от неё выгода ..."
4,1,часть 1 статья 158 похитил телефон
...,...,...
237774,3,елку нарядили? =)
237775,1,"Имеется переработка при 75% ставки, отгулы не ..."
237776,4,Попробовала варить рис с половиной кубика для ...
237777,4,Почему рекоменд... Почему рекомендуют есть фру...


# Читалка данных

## Что происходит ниже
1. Мы задаем x_data, y_data (таргеты), word2index (маппер из слова в индекс слова), sequence_length (максимальная длина последовательности, если больше, ограничить ею), pad_token (токен паддинга и задаем его индекс pad_index).
1. Загружаем данные:
    1. Проходимся по датасету
    1. Предобрабатываем каждый текст в датасете
    1. Индексируем его
    1. Паддим до нужной длины
1. Когда нам нужно достать пример из датасета мы берем индексированный ```x``` и соответствующий этому индексу ```y```, наш ```x``` также паддим (или ограничиваем длину) и переводим в ```torch.Tensor(x).long()```. Для ```y``` этого делать не потребуется, в dataloader'е таргеты преобразуются в тензор сами.


In [94]:
import torch

In [95]:
class WordData(Dataset):
    
    def __init__(self, x_data, y_data, word2index, sequence_length=32, pad_token='PAD', verbose=True):
        
        super().__init__()
        
        self.x_data = []
        self.y_data = y_data
        
        self.word2index = word2index
        self.sequence_length = sequence_length
        
        self.pad_token = pad_token
        self.pad_index = self.word2index[self.pad_token]
        
        self.load(x_data, verbose=verbose)
        
    @staticmethod
    def process_text(text):
        
        # Место для вашей предобработки
        
        words = [token for token in word_tokenize(text.lower()) if token not in stops] #  wordpunct_tokenize(text.lower())
        #words = re.findall('[a-яА-ЯеЁ]+', text.lower())
        return words
        
    def load(self, data, verbose=True):
        
        data_iterator = tqdm(data, desc='Loading data', disable=not verbose)
        
        for text in data_iterator:
            
            words = self.process_text(text)
            
            indexed_words = self.indexing(words)
            
            self.x_data.append(indexed_words)
    
    def indexing(self, tokenized_text):

        # здесь мы не используем токен UNK, потому что мы его специально не учили
        # становится непонятно какой же эмбеддинг присвоить неизвестному слову,
        # поэтому просто выбрасываем наши неизветсные слова
        
        return [self.word2index[word] for word in tokenized_text if word in self.word2index]
    
    def padding(self, sequence):
        
        # Ограничить длину self.sequence_length
        # если длина меньше максимально - западить
        if len(sequence)< self.sequence_length:
          add_pad = self.sequence_length - len(sequence)
          return sequence+[self.pad_index]*add_pad
        else:
          return sequence[:self.sequence_length]
    
    def __len__(self):
        
        return len(self.x_data)
    
    def __getitem__(self, idx):
        
        x = self.x_data[idx]
        x = self.padding(x)
        x = torch.Tensor(x).long()
        
        y = self.y_data[idx]
        
        return x, y

In [96]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [97]:
x_train, x_validation, y_train, y_validation = train_test_split(data.text, data.category, test_size=0.1)

train_dataset = WordData(list(x_train), list(y_train), word2index)
train_loader = DataLoader(train_dataset, batch_size=64)

validation_dataset = WordData(list(x_validation), list(y_validation), word2index)
validation_loader = DataLoader(validation_dataset, batch_size=64)

Loading data: 100%|██████████| 23778/23778 [00:04<00:00, 5712.02it/s]


In [98]:
for x, y in train_loader:
    break

In [99]:
x

tensor([[ 40156,   2228,   1469,  ...,      0,      0,      0],
        [  1657,   1255,    847,  ...,      0,      0,      0],
        [  3240,  12186,    498,  ...,      0,      0,      0],
        ...,
        [  5859,  38587,  72203,  ...,      0,      0,      0],
        [  1704,   3214,  67633,  ...,      0,      0,      0],
        [  6831, 107499,  10396,  ...,      0,      0,      0]])

In [100]:
y

tensor([3, 4, 3, 2, 1, 1, 1, 2, 2, 1, 0, 2, 1, 1, 0, 1, 1, 4, 0, 1, 0, 4, 1, 1,
        4, 1, 1, 2, 1, 0, 1, 0, 1, 1, 1, 3, 4, 1, 4, 3, 4, 1, 2, 4, 0, 0, 2, 1,
        1, 1, 4, 4, 1, 0, 4, 1, 4, 1, 3, 3, 4, 4, 4, 1])

# Обучить нейронку

In [101]:
from math import sqrt

class model_with_att(torch.nn.Module):
  def __init__(self, matrix_w, n, embedding_dim, hidden_size, n_layers=2): #n - количетсво категорий   ,...
        
        super().__init__()

        self.n = n
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        # . . . 

        self.emb_layer = torch.nn.Embedding.from_pretrained(torch.Tensor(matrix_w))

        self.LSTM = torch.nn.LSTM(embedding_dim, 
                            hidden_size, 
                            n_layers, 
                            bidirectional=True) # задайте лстм, можно 2 уровня, лучше бидирекциональный, в доке торча есть инофрмация как это сделать в одну строчку
        
        self.q_proj = torch.nn.Linear(hidden_size * 2, hidden_size  * 2)  # три линейных преобразования, размерность совпадает с выходом из лстм (если БИлстм то надо умножить ее на 2)
        self.k_proj = torch.nn.Linear(hidden_size * 2, hidden_size * 2)
        self.v_proj = torch.nn.Linear(hidden_size * 2, hidden_size * 2)

        self.att_soft = torch.nn.Softmax(dim = 2)
        
        self.cnn_3gr = torch.nn.Conv1d(in_channels=hidden_size * 2, out_channels=80, kernel_size=3, padding='same') # три конволюционных фильтра с разными ядрами (3,4,5) чтобы были всякие нграммы ловить
        self.cnn_4gr = torch.nn.Conv1d(in_channels=hidden_size * 2, out_channels=80, kernel_size=4, padding='same') # проверить out_channels
        self.cnn_5gr = torch.nn.Conv1d(in_channels=hidden_size * 2, out_channels=80, kernel_size=5, padding='same')

        self.linear_1 = torch.nn.Linear(in_features=240, out_features=64)# сверху накидываем два полносвязных слоя для классификации ДОПИСАТЬ!
        self.relu = torch.nn.ReLU()
        self.linear_2 = torch.nn.Linear(in_features=64, out_features=n)  # ДОПИСАТЬ!

        
  def forward(self, x):
      x_emb = self.emb_layer(x)  # примените эмбеддинги
      x_emb = x_emb.transpose(0, 1) # транспонируйте тензор для лстм как было описано выше
      x, _ = self.LSTM(x_emb) # применим лстм, не забываем что на выходе у него много всяких последовательностей, нам нужна только эта
      x = x.transpose(0, 1)
      # транспонируйте обратно

      x_q = self.q_proj(x) #применим линейные преобразования для селф-эттеншена
      x_k = self.k_proj(x)
      x_v = self.v_proj(x)

      att_scores = torch.bmm(x_q, x_k.transpose(-2, -1)) / sqrt(x_q.size(-1))
      # посмотрите в презентацию и перемножьте нужные тензора изспольуя функцию bmm из торча, перед этим одну из матриц обзательно транспонируйте
      # результат обязательно поделите на корень из последней размерности (то есть на рземер эмбеддинга из предыдущего слоя)
      att_dist = self.att_soft(att_scores) # накидываем софтмакс
      attention_vectors = torch.bmm(att_dist, x_v)  # тут тоже что то с чем то нужно перемножить :)

      x_att = attention_vectors.transpose(2,1) #транспонируем для конфолючионнах фильтров

      x_cnn3 = self.cnn_3gr(x_att)
      x_cnn4 = self.cnn_4gr(x_att)
      x_cnn5 = self.cnn_5gr(x_att)

      frst, _ =  x_cnn3.max(dim= -1,) # cделаем макс пуллинг
      sc, _ = x_cnn4.max(dim= -1,)
      thr, _ = x_cnn5.max(dim= -1,)
      
      x_cat = torch.cat((frst, sc, thr), dim=-1) # а теперь объединим результаты
      
      x = self.linear_1(x_cat)  # пару полносвязных слоев с релу для классификации
      x = self.relu(x)    
      x = self.linear_2(x)
    
      return x

In [102]:
n_classes = data.category.unique().shape[0]

In [103]:
model = model_with_att(vectors, n_classes, embedding_dim=300, hidden_size=128)

In [104]:
model #если сделать batch_first=True, то можно не транспонировать батчи

model_with_att(
  (emb_layer): Embedding(119903, 300)
  (LSTM): LSTM(300, 128, num_layers=2, bidirectional=True)
  (q_proj): Linear(in_features=256, out_features=256, bias=True)
  (k_proj): Linear(in_features=256, out_features=256, bias=True)
  (v_proj): Linear(in_features=256, out_features=256, bias=True)
  (att_soft): Softmax(dim=2)
  (cnn_3gr): Conv1d(256, 80, kernel_size=(3,), stride=(1,), padding=same)
  (cnn_4gr): Conv1d(256, 80, kernel_size=(4,), stride=(1,), padding=same)
  (cnn_5gr): Conv1d(256, 80, kernel_size=(5,), stride=(1,), padding=same)
  (linear_1): Linear(in_features=240, out_features=64, bias=True)
  (relu): ReLU()
  (linear_2): Linear(in_features=64, out_features=5, bias=True)
)

In [105]:
with torch.no_grad():
    pred = model(x)

In [106]:
pred.shape

torch.Size([64, 5])

In [107]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [108]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters())

model = model.to(device)
criterion = criterion.to(device)

In [109]:
epochs = 10
losses = []
best_test_loss = 10.

test_f1 = []

for n_epoch in range(epochs):
    
    train_losses = []
    test_losses = []
    test_targets = []
    test_pred_class = []
    
    progress_bar = tqdm(total=len(train_loader.dataset), desc='Epoch {}'.format(n_epoch + 1))
    
    model.train()
    
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        
        pred = model(x)
        loss = criterion(pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        train_losses.append(loss.item())
        losses.append(loss.item())
        
        progress_bar.set_postfix(train_loss = np.mean(losses[-500:]))

        progress_bar.update(x.shape[0])
        
    progress_bar.close()
    
    model.eval()
    
    for x, y in validation_loader:
        
        x = x.to(device)

        with torch.no_grad():

            pred = model(x)

            pred = pred.cpu()

            test_targets.append(y.numpy())
            test_pred_class.append(np.argmax(pred, axis=1))

            loss = criterion(pred, y)

            test_losses.append(loss.item())
        
    mean_test_loss = np.mean(test_losses)

    test_targets = np.concatenate(test_targets).squeeze()
    test_pred_class = np.concatenate(test_pred_class).squeeze()

    f1 = f1_score(test_targets, test_pred_class, average='micro')

    test_f1.append(f1)
    
    print()
    print('Losses: train - {:.3f}, test - {:.3f}'.format(np.mean(train_losses), mean_test_loss))

    print('F1 test - {:.3f}'.format(f1))
        
    # Early stopping:
    if mean_test_loss < best_test_loss:
        best_test_loss = mean_test_loss
    else:
        print('Early stopping')
        break

Epoch 1: 100%|██████████| 214001/214001 [00:48<00:00, 4402.59it/s, train_loss=0.485]



Losses: train - 0.573, test - 0.488
F1 test - 0.825


Epoch 2: 100%|██████████| 214001/214001 [00:48<00:00, 4393.55it/s, train_loss=0.452]



Losses: train - 0.465, test - 0.454
F1 test - 0.839


Epoch 3: 100%|██████████| 214001/214001 [00:48<00:00, 4429.86it/s, train_loss=0.431]



Losses: train - 0.440, test - 0.447
F1 test - 0.843


Epoch 4: 100%|██████████| 214001/214001 [00:48<00:00, 4431.18it/s, train_loss=0.414]



Losses: train - 0.421, test - 0.446
F1 test - 0.842


Epoch 5: 100%|██████████| 214001/214001 [00:48<00:00, 4447.89it/s, train_loss=0.397]



Losses: train - 0.403, test - 0.455
F1 test - 0.839
Early stopping


Если вы запускаете много раз колаб окна и ткдм начинает беситься, можно запустить окно ниже, ткдм обновится и все снова станет хорошо

In [129]:
for instance in list(tqdm._instances): 
    tqdm._decr_instances(instance)

Качество незначительно выросло (с 0,843 до 0,845). Я добавил `dropout` в `LSTM`, заменил `ReLU` на `LeakyReLU`, увеличил `out_channels` в свёрточных слоях до 120.

In [134]:
from math import sqrt

class model_with_att(torch.nn.Module):
  def __init__(self, matrix_w, n, embedding_dim, hidden_size, n_layers=2): #n - количетсво категорий   ,...
        
        super().__init__()

        self.n = n
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        # self.dropout = torch.nn.Dropout(p=0.2)
        # . . . 

        self.emb_layer = torch.nn.Embedding.from_pretrained(torch.Tensor(matrix_w))

        self.LSTM = torch.nn.LSTM(embedding_dim, 
                            hidden_size, 
                            n_layers,
                            dropout=0.1,
                            bidirectional=True) # задайте лстм, можно 2 уровня, лучше бидирекциональный, в доке торча есть инофрмация как это сделать в одну строчку
        
        self.q_proj = torch.nn.Linear(hidden_size * 2, hidden_size  * 2)  # три линейных преобразования, размерность совпадает с выходом из лстм (если БИлстм то надо умножить ее на 2)
        self.k_proj = torch.nn.Linear(hidden_size * 2, hidden_size * 2)
        self.v_proj = torch.nn.Linear(hidden_size * 2, hidden_size * 2)

        self.att_soft = torch.nn.Softmax(dim = 2)
        
        self.cnn_3gr = torch.nn.Conv1d(in_channels=hidden_size * 2, out_channels=120, kernel_size=3, padding='same') # три конволюционных фильтра с разными ядрами (3,4,5) чтобы были всякие нграммы ловить
        self.cnn_4gr = torch.nn.Conv1d(in_channels=hidden_size * 2, out_channels=120, kernel_size=4, padding='same') # проверить out_channels
        self.cnn_5gr = torch.nn.Conv1d(in_channels=hidden_size * 2, out_channels=120, kernel_size=5, padding='same')

        self.linear_1 = torch.nn.Linear(in_features=360, out_features=64)# сверху накидываем два полносвязных слоя для классификации ДОПИСАТЬ!
        # self.relu = torch.nn.ReLU()
        self.lrelu = torch.nn.LeakyReLU()
        self.linear_2 = torch.nn.Linear(in_features=64, out_features=n)  # ДОПИСАТЬ!

        
  def forward(self, x):
      x_emb = self.emb_layer(x)  # примените эмбеддинги
      x_emb = x_emb.transpose(0, 1) # транспонируйте тензор для лстм как было описано выше
      x, _ = self.LSTM(x_emb) # применим лстм, не забываем что на выходе у него много всяких последовательностей, нам нужна только эта
      x = x.transpose(0, 1)
      # транспонируйте обратно

      x_q = self.q_proj(x) #применим линейные преобразования для селф-эттеншена
      x_k = self.k_proj(x)
      x_v = self.v_proj(x)

      att_scores = torch.bmm(x_q, x_k.transpose(-2, -1)) / sqrt(x_q.size(-1))
      # посмотрите в презентацию и перемножьте нужные тензора изспольуя функцию bmm из торча, перед этим одну из матриц обзательно транспонируйте
      # результат обязательно поделите на корень из последней размерности (то есть на рземер эмбеддинга из предыдущего слоя)
      att_dist = self.att_soft(att_scores) # накидываем софтмакс
      attention_vectors = torch.bmm(att_dist, x_v)  # тут тоже что то с чем то нужно перемножить :)

      x_att = attention_vectors.transpose(2,1) #транспонируем для конфолючионнах фильтров

      x_cnn3 = self.cnn_3gr(x_att)
      x_cnn4 = self.cnn_4gr(x_att)
      x_cnn5 = self.cnn_5gr(x_att)

      frst, _ =  x_cnn3.max(dim= -1,) # cделаем макс пуллинг
      sc, _ = x_cnn4.max(dim= -1,)
      thr, _ = x_cnn5.max(dim= -1,)
      
      x_cat = torch.cat((frst, sc, thr), dim=-1) # а теперь объединим результаты
      
      x = self.linear_1(x_cat)  # пару полносвязных слоев с релу для классификации
      x = self.lrelu(x)    
      x = self.linear_2(x)
    
      return x

In [135]:
model = model_with_att(vectors, n_classes, embedding_dim=300, hidden_size=128)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters())

model = model.to(device)
criterion = criterion.to(device)

In [136]:
epochs = 10
losses = []
best_test_loss = 10.

test_f1 = []

for n_epoch in range(epochs):
    
    train_losses = []
    test_losses = []
    test_targets = []
    test_pred_class = []
    
    progress_bar = tqdm(total=len(train_loader.dataset), desc='Epoch {}'.format(n_epoch + 1))
    
    model.train()
    
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        
        pred = model(x)
        loss = criterion(pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        train_losses.append(loss.item())
        losses.append(loss.item())
        
        progress_bar.set_postfix(train_loss = np.mean(losses[-500:]))

        progress_bar.update(x.shape[0])
        
    progress_bar.close()
    
    model.eval()
    
    for x, y in validation_loader:
        
        x = x.to(device)

        with torch.no_grad():

            pred = model(x)

            pred = pred.cpu()

            test_targets.append(y.numpy())
            test_pred_class.append(np.argmax(pred, axis=1))

            loss = criterion(pred, y)

            test_losses.append(loss.item())
        
    mean_test_loss = np.mean(test_losses)

    test_targets = np.concatenate(test_targets).squeeze()
    test_pred_class = np.concatenate(test_pred_class).squeeze()

    f1 = f1_score(test_targets, test_pred_class, average='micro')

    test_f1.append(f1)
    
    print()
    print('Losses: train - {:.3f}, test - {:.3f}'.format(np.mean(train_losses), mean_test_loss))

    print('F1 test - {:.3f}'.format(f1))
        
    # Early stopping:
    if mean_test_loss < best_test_loss:
        best_test_loss = mean_test_loss
    else:
        print('Early stopping')
        break

Epoch 1: 100%|██████████| 214001/214001 [00:58<00:00, 3635.24it/s, train_loss=0.486]



Losses: train - 0.566, test - 0.489
F1 test - 0.825


Epoch 2: 100%|██████████| 214001/214001 [01:00<00:00, 3540.94it/s, train_loss=0.456]



Losses: train - 0.466, test - 0.457
F1 test - 0.837


Epoch 3: 100%|██████████| 214001/214001 [00:58<00:00, 3643.18it/s, train_loss=0.435]



Losses: train - 0.443, test - 0.447
F1 test - 0.842


Epoch 4: 100%|██████████| 214001/214001 [00:58<00:00, 3638.94it/s, train_loss=0.417]



Losses: train - 0.424, test - 0.444
F1 test - 0.845


Epoch 5: 100%|██████████| 214001/214001 [00:58<00:00, 3630.16it/s, train_loss=0.4]



Losses: train - 0.407, test - 0.445
F1 test - 0.844
Early stopping


# Оценка
1. Добрались сюда - очень хорошо - получилась такая же точность или около того - 7 баллов.
2. Поставили эксперименты и повысили точность относительно своей и не ниже F1 test - 0.841 - 8 баллов.
3. Запустили бертовую тетрадку и разобрались. Получился сравнимый результат - 10 баллов 